In [14]:
%matplotlib inline
import sys
sys.path.append('..')
from modules.backtesting import BackTesting
import pandas as pd
from keras.models import load_model
from modules.factory import norm, fmeasure, recall
import h5py
pd.options.display.float_format = '{:,.2f}'.format #화면에 소숫점 2번째 자리까지만 표시

### 0. Parameters

In [12]:
predic_model_file= '../data/keras models/uptrend-predic(20-240-40).h5'
raw_data_file = '../data/training set/daily-future-40day-trend.h5'

In [15]:
predic_model = load_model(predic_model_file, custom_objects={'recall':recall, 'fmeasure':fmeasure})
f = h5py.File(raw_data_file, 'r')
FILES = [i[1] for i in f.items()]

### 1. 전략구현

In [16]:
def strat(report):
    data = report.data
    data['ma20'] = data.Settle.rolling(10).mean()
    data['ma60'] = data.Settle.rolling(60).mean()
    data['cross'] = (data.ma20 > data.ma60).astype('int')
    data['signal'] = data.cross.diff().shift(1)
    data.dropna(inplace=True)
    signals = data[(data.signal == 1) | (data.signal == -1)]
        
    for date, price, signal in signals[['Open','signal']].itertuples():
        if signal == 1:
            report.buy('Long', date, price)
        elif signal == -1:
            report.sell(date, price)